## Pure Photonics Driver

### Import Dependencies

In [2]:
import pyvisa

### Define Registers

In [3]:
REG_Nop=0x00
REG_Mfgr=0x02
REG_Model=0x03
REG_Serial=0x04
REG_Release=0x06
REG_Gencfg=0x08
REG_AeaEar=0x0B
REG_Iocap=0x0D
REG_Ear=0x10
REG_Dlconfig=0x14
REG_Dlstatus=0x15
REG_Channel=0x30
REG_Power=0x31
REG_Resena=0x32
REG_Grid=0x34
REG_Fcf1=0x35
REG_Fcf2=0x36
REG_LF1=0x40
REG_LF2=0x41
REG_Oop=0x42
REG_Opsl=0x50
REG_Opsh=0x51
REG_Lfl1=0x52
REG_Lfl2=0x53
REG_Lfh1=0x54
REG_Lfh2=0x55
REG_Currents=0x57
REG_Temps=0x58
REG_Ftf=0x62
REG_Channelh=0x65
REG_Mode=0x90
REG_PW=0xE0
REG_Csweepsena=0xE5
REG_Csweepamp=0xE4
REG_Cscanamp=0xE4
REG_Cscanon=0xE5
REG_Csweepon=0xE5
REG_Csweepoffset=0xE6
REG_Cscanoffset=0xE6
REG_Cscansled=0xF0
REG_Cscanf1=0xF1
REG_Cscanf2=0xF2
REG_CjumpTHz=0xEA
REG_CjumpGHz=0xEB
REG_CjumpSled=0xEC
REG_Cjumpon=0xED
REG_Cjumpoffset=0xE6

### Define Functions (Driver)

In [4]:
def send_command(lsr, register, data=None, signed=False):
    """Sends commands to a device.
    This function takes the hexstring, turns it into a bytestring,
    and writes it to the device.
    This function should probably be hidden from the user.
    :param device: Should be a Serial object that you can write to.
    :param hexstring: a hexstring to send to the device
    :returns: nothing
    """
    write = (data is not None)
 
    # convert to register to a bytestring
    register_bytes = register.to_bytes(1, 'big')
 
    # convert data to bytestring
    if write:
        data_bytes = data.to_bytes(2, 'big', signed=signed)
 
    else:
        data_bytes = (0).to_bytes(2, 'big')
 
    # compute the checksum
    checksum = compute_checksum(
        (write.to_bytes(1, 'big') + register_bytes + data_bytes).hex()
    )
 
    # compute and convery header to bytestring
    header = checksum * 16 + write
    print(header)
    header_bytes = header.to_bytes(1, 'big')
    print(header_bytes)
 
    # form full command and send.
    command = header_bytes + register_bytes + data_bytes
    print(command)
    lsr.write_raw(command)
 
def compute_checksum(hexstring):
    """ Computes the command checksum
    :param register: the register to write to
    :param data: the data to write the register
    :param write: whether or not you are writing to the register
    :returns: the checksum value
    """
    # get the hexstring for the command without the
 
    byte_list = bytes.fromhex(hexstring)
 
    bip8 = byte_list[0] & 15 ^ byte_list[1] ^ byte_list[2] ^ byte_list[3]
 
    return (bip8 & 240) >> 4 ^ bip8 & 15

 
def get_response(lsr, register):
    """This function should read from self._device. This should be called
 
    :param register:
    :returns: ???
 
    """
    response = lsr.read_bytes(4)
 
    print(f'response: {response.hex()}')
 
    # get the checksum and ... check it.
    checksum = int(response.hex()[0], 16)
    computed_checksum = compute_checksum(response.hex())
    print(checksum, computed_checksum)
 
    status = int(f'{response[0]:08b}'[-2:], 2)
    print(f'status: {status}')
 
    if status==2:
        response = read_aea(lsr)
        return response
    # try:
    #     raise _response_status[status]
 
    # except AEAException:
    #     response = read_aea()
    #     return response
 
    return response[2:]
 
def nop(lsr, data=None):
        """The No-Op operation.
 
        This is a good test to see if your laser is communicating properly.
        It should read 0000 data or echo whatever data you send it if you send it something.
        The data you write to nop gets overwritten as soon as you read it back.
 
        `nop()` also returns more informative errors following an ExecutionError.
        This is not called by default so you must do this explicitly if you want
        to see more informative error information.
 
        :param data: Data to write
 
        """
        # pretty sure the data does nothing
        if data is not None:
            response = lsr._nop(data)
        else:
            response = lsr._nop()
 
        error_field = int(response.hex()[-1], 16)
        if bool(error_field):
            raise _nop_errors[error_field]

### Main Routine

In [7]:
rm = pyvisa.ResourceManager()
print(rm.list_resources())
lsr=rm.open_resource('ASRL12::INSTR', baud_rate=9600)
 
# print("before Nop and send command")
send_command(lsr, REG_Nop, None, False)
x=get_response(lsr, REG_Nop)
 
# laser enable
data = [0] * 16
data[3] = 1
data = int(''.join(str(x) for x in data[::-1]), 2)
send_command(lsr,REG_Resena,data,False)
x=get_response(lsr, REG_Resena)

# set frequency
send_command(lsr, REG_LF1, None, False)
x=get_response(lsr, REG_LF1)
send_command(lsr, REG_LF2, None, False)
x=get_response(lsr, REG_LF2)
 
# get optical power
send_command(lsr,REG_Oop,None,False)
x=get_response(lsr, REG_LF2)
 

('TCPIP0::10.10.60.153::inst0::INSTR', 'TCPIP0::10.10.60.3::inst0::INSTR', 'ASRL9::INSTR', 'ASRL12::INSTR')
0
b'\x00'
b'\x00\x00\x00\x00'
response: 54000010
5 5
status: 0
129
b'\x81'
b'\x812\x00\x08'
response: f7320108
15 15
status: 3
64
b'@'
b'@@\x00\x00'
response: 544000be
5 5
status: 0
80
b'P'
b'PA\x00\x00'
response: 444103e8
4 4
status: 0
96
b'`'
b'`B\x00\x00'
response: 6442f830
6 6
status: 0


In [8]:
# set channel to 56
channel=0x00000038
channel_hex = f'{channel:08x}'
channell = int(channel_hex[4:], 16)
channelh = int(channel_hex[0:4], 16)
 
send_command(lsr, REG_Channel, channell, False)
x=get_response(lsr, REG_Channel)
 
send_command(lsr, REG_Channelh, channelh, False)
x=get_response(lsr, REG_Channelh)

145
b'\x91'
b'\x910\x008'
response: 57300100
5 5
status: 3
33
b'!'
b'!e\x00\x00'
response: 74650000
7 7
status: 0


### End Session/Close laser handle

In [9]:
# disable laser
send_command(lsr,REG_Resena,0,False)
x=get_response(lsr, REG_Resena)
lsr.close()

1
b'\x01'
b'\x012\x00\x00'
response: 54320000
5 5
status: 0
